# Começando a pensar no projeto


## Contextualização

A PyCoders Ltda., cada vez mais especializada no mundo da Inteligência Artificial e Ciência de Dados, foi procurada por uma fintech para desenvolver um projeto de teste de produtos. Nesse projeto, espera-se a criação de valor que **discrimine ao máximo** os **produtos com falhas** dos **produto sem falhas**. 

Para cada product_code, você recebe vários atributos de produto (fixados para o código), bem como vários valores de medição para cada produto individual, representando vários métodos de teste de laboratório. Cada produto é usado em um experimento simulado de ambiente do mundo real e absorve uma certa quantidade de fluido (carregamento) para ver se falha ou não.

Sua tarefa é usar os dados para prever falhas de novos códigos de produtos individuais com seus resultados de testes de laboratório individuais.

Para isso, foi disponibilizada uma base de dados com milhares de casos de **testes do passado** com diversas características dos produtos. 

Entrega: um modelo com a **melhor performance** possível.

Métrica de performance (inicialmente proposta): **ROC-AUC** (mas isso é flexível, conforme detalhado mais abaixo!)

## Base de Dados

Serão utilizadas bases de dados com **informações de diversos produtos**. 

O conjunto de dados está dividido em **treino e teste**, todos no formato csv. 

Toda a modelagem, validação e avaliação deve ser feita em cima do conjunto de **treino**, que contém o target (arquivo: `train_data.csv`)

## INICIANDO O PROJETO

**Carregando as bibliotecas**

In [116]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# ANÁLISE E EXPLORAÇÃO DOS DADOS

In [117]:
#ler os datasets
train_data_read = pd.read_csv('train_data.csv')
test_data_read = pd.read_csv('test_data.csv')

In [118]:
#Avaliar a quantidade de linhas dos dados de treino
train_data_read.shape

(21256, 27)

In [119]:
##Avaliar a quantidade de linhas dos dados de teste
test_data_read.shape

(5314, 27)

In [120]:
#Verificar o quão desbalanceado está o target
train_data_read['failure'].value_counts()

0    16737
1     4519
Name: failure, dtype: int64

Como é notado acima, o target está muita desbalanceado. Dessa forma é necessário aplicar algumas técnicas de balanceamento e comparar os resultados.

In [121]:
#Analisando o dataset de treino
train_data_read.head()

,Unnamed: 0,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,2596,2596,A,94.34,material_7,material_8,9,5,6,2,...,10.625,NaN,15.828,12.402,13.289,15.886,15.990,15.352,803.056,1
1,23021,23021,E,117.02,material_7,material_6,6,9,11,10,...,10.941,13.620,17.654,11.603,14.949,18.139,13.940,NaN,719.968,0
2,10202,10202,B,256.42,material_5,material_5,8,8,5,10,...,10.620,16.771,17.423,NaN,16.413,14.752,16.220,16.671,661.213,1
3,19048,19048,D,87.76,material_7,material_5,6,6,5,13,...,10.387,18.028,19.016,11.727,16.378,17.244,15.472,14.017,536.760,0
4,14789,14789,C,97.84,material_7,material_8,5,8,17,6,...,13.372,15.236,19.875,9.911,16.082,18.070,15.504,15.130,535.931,1


Percebe-se a principio que as colunas 'Unnamed: 0' e id tem os mesmo valores. Dessa forma é necessário uma análise mais minuciosa para confirmar tal hipótese.

In [122]:
#Verifcar quantidade de valores da coluna 'Unnamed: 0'
len(train_data_read['Unnamed: 0'].unique().tolist())

21256

In [123]:
#Verifcar quantidade de valores da coluna id
len(train_data_read['id'].unique().tolist())

21256

In [124]:
#Comparar os valores das duas colunas
comparation_id_column = pd.DataFrame(train_data_read['Unnamed: 0'] == train_data_read['id'], columns=['Result'])

In [125]:
#Buscar por registros onde a comparação foi falsa
comparation_id_column.query('Result == False')

,Result


In [126]:
#Como esperado as duas primeiras colunas são iguais e são colunas de indexação. 
#Sendo assim ambas serão excluídas.
train_data_read.drop(axis=1, columns=['Unnamed: 0', 'id'], inplace=True)
test_data_read.drop(axis=1, columns=['Unnamed: 0', 'id'], inplace=True)

Nota-se também que algumas colunas são categóricas, dessa forma é necessário trata-lás previamente. Mais a frente será aplicado algumas técnicas para lidar com dados categóricos. 

Outra análise importante é sobre a quantidade de valores nulos. Esse caso também requer algum tipo de tratamento para melhorar a qualidade dos dados sem a necessidade de excluir todo o registro.

Aqui observa-se que existem algumas colunas com alguns dados nulos.
Porém nem todas as features serão relevantes para o modelo.
Para otimizar o volume das features utilizadas no treinamento será avaliado o valor das suas correlações logo abaixo

print_isna_sum()

In [127]:
# plt.figure(figsize=(20, 14))
# sns.heatmap(train_data_read.corr(),
#             annot = True,
#             fmt = '.2f',
#             cmap='Blues')
# plt.title('Correlação entre as features')
# plt.show()

Devido a baixa correlação que grande parte das features apresentam, foi mantida apenas aquelas com correlação maior que 0.3

In [128]:
#DataFrame de treino resultante após exclusão das features com baixa correlação
train_data_without_low_correlation_features = train_data_read.drop(axis=1, 
                                        columns=[
                                            'loading',
                                            'measurement_3',
                                            'measurement_4',
                                            'measurement_5',
                                            'measurement_6',
                                            'measurement_7',
                                            'measurement_8',
                                            'measurement_9',
                                            'measurement_13'],
                                        inplace=False)

In [129]:
#DataFrame de teste resultante após exclusão das features com baixa correlação
test_data_without_low_correlation_features = test_data_read.drop(axis=1, 
                                        columns=[
                                           'loading',
                                            'measurement_3',
                                            'measurement_4',
                                            'measurement_5',
                                            'measurement_6',
                                            'measurement_7',
                                            'measurement_8',
                                            'measurement_9',
                                            'measurement_13'], inplace=False)

Escolhida as colunas que serão relevantes para o modelo, inicia-se o tratamento com os dados já separados. Como foi fornecido um arquivo separado com os dados de teste não será aplicado o train_test_split nesse caso.

In [130]:
#Setando as features de treino
X_train = train_data_without_low_correlation_features.drop(columns=['failure'])

#Setando o target de treino
y_train = train_data_without_low_correlation_features['failure']

In [131]:
#Setando as features de teste
X_test = test_data_without_low_correlation_features.drop(columns=['failure'])

#Setando o target de teste
y_test = test_data_without_low_correlation_features['failure']

# Preprocessamento de dados -> FEATURE ENGINEERING

In [132]:
#Primeramente é necessário separar as colunas categóricas das numéricas.
#Uma das formas utilizadas é aplicando a filtragem pelo tipo da coluna com o método select_dtypes

# Colunas categoricas
categorical_columns = list(X_train.select_dtypes('object').columns)

# Colunas numéricas
numerical_columns = list(X_train.select_dtypes('number').columns)

In [133]:
#Definida o conjunto de colunas pelo tipo, o próximo passo é definir as tratativas do pipeline.
#Nesse caso os valores nulos serão substituídos pela string # e posteriormente os dados serão transformados
#em valores numéricos utilizando a técnica de OneHotEncoder
categorical_pipeline = Pipeline([ 
     ('impute', SimpleImputer(strategy='constant', fill_value='#')),
     ('encode', OneHotEncoder(handle_unknown='ignore', sparse=False, drop='first'))
])
categorical_pipeline

Pipeline(steps=[('impute', SimpleImputer(fill_value='#', strategy='constant')),
                ('encode',
                 OneHotEncoder(drop='first', handle_unknown='ignore',
                               sparse=False))])

In [134]:
#Pipeline para tratamento numérico. Nesse caso foi escolhido a estratégia de preenchimento pela média
#e MinMaxScaler como método de escalonamento.
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler()) 
])

In [135]:
#Como será aplicado tratamento separados para as colunas do tipo categórica e numérica
#será utilizado a técnica de ColumnTransformer. Que permite criar um pipeline com tratamentos
#distintos por conjunto de colunas.
column_transformer_pipeline = ColumnTransformer([
    ('cat', categorical_pipeline, categorical_columns),
    ('num', numeric_pipeline, numerical_columns)
])

In [136]:
#Aplicado o fit nos dados de treino
column_transformer_pipeline.fit(X_train)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(fill_value='#',
                                                                strategy='constant')),
                                                 ('encode',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['product_code', 'attribute_0',
                                  'attribute_1']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', MinMaxScaler())]),
                                 ['attribute_2', 'attribute_3', 'measurement_0',
                                  'measurement_1', 'measurement_2',
                                  'measurement_10', 'measurement_11',
                                  'measurement_12', 'measurement_14',
                                  'measurement_15', 'measurement_16',
                                  'measurement_17'])])

In [137]:
#Para visualisar os dados após o tratamento é necessário transformá-los novamente em um DataFrame.
#Como o método do OneHotEncoder cria novas colunas é preciso antes obter o nome das novas colunas.

#lista com o nome das colunas categóricas que foram utilizadas
categorical_transform_names = column_transformer_pipeline.named_transformers_['cat']['encode'].get_feature_names_out(categorical_columns)

#licas com o nome de todas as colunas.
#Como não foi aplicado oneHotEncoder nas colunas numéricas podemos utilizar a mesma lista anterior.
columns_transform_name = np.append(categorical_transform_names, numerical_columns)

X_train_reload_df = pd.DataFrame(column_transformer_pipeline.transform(X_train), columns=columns_transform_name)

In [138]:
X_train_reload_df.head()

,product_code_B,product_code_C,product_code_D,product_code_E,attribute_0_material_7,attribute_1_material_6,attribute_1_material_8,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_10,measurement_11,measurement_12,measurement_14,measurement_15,measurement_16,measurement_17
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.00,0.00,0.206897,0.068966,0.125000,0.517026,0.255482,0.578513,0.527031,0.549912,0.392621,0.543248
1,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.25,1.00,0.379310,0.344828,0.291667,0.326619,0.394036,0.514501,0.703047,0.386200,0.468283,0.468797
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.75,0.172414,0.344828,0.291667,0.566130,0.376508,0.524435,0.438438,0.568280,0.484263,0.416150
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.25,0.25,0.172414,0.448276,0.416667,0.661675,0.497382,0.524435,0.633125,0.508545,0.299868,0.304633
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.00,0.75,0.586207,0.206897,0.125000,0.449453,0.562562,0.378946,0.697656,0.511100,0.377197,0.303891


In [139]:
#Transformado os dados cria-se o pipeline de treinamento
train_pipeline = Pipeline([
    ('transformer_pipe', column_transformer_pipeline),
    ('dt', DecisionTreeClassifier(random_state=1))
])

Como foi avaliado inicialmente, os valores do target estão desbalanceados. Para minizar o impacto dessa diferença pode-se aplicar algumas técnicas para lidar com essa situação. Nesse caso será aplicado a técnica de SMOTE, aumentando a quantidade de amostras da classe minoritária para igualar com a majoritária

In [140]:
#Quantidade de registros pré SMOTE
print('Original y_train shape', Counter(y_train))

Original y_train shape Counter({0: 16737, 1: 4519})


In [141]:
#Balanceamento do dado
sm = SMOTE(random_state=42)
X_train_resample, y_train_resample = sm.fit_resample(X_train_reload_df, y_train)

In [142]:
#Quantidade de registros pós SMOTE
print('oversampling y_train shape', Counter(y_train_resample))

oversampling y_train shape Counter({1: 16737, 0: 16737})


In [143]:
X_train_resample.head()

,product_code_B,product_code_C,product_code_D,product_code_E,attribute_0_material_7,attribute_1_material_6,attribute_1_material_8,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_10,measurement_11,measurement_12,measurement_14,measurement_15,measurement_16,measurement_17
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.00,0.00,0.206897,0.068966,0.125000,0.517026,0.255482,0.578513,0.527031,0.549912,0.392621,0.543248
1,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.25,1.00,0.379310,0.344828,0.291667,0.326619,0.394036,0.514501,0.703047,0.386200,0.468283,0.468797
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.75,0.172414,0.344828,0.291667,0.566130,0.376508,0.524435,0.438438,0.568280,0.484263,0.416150
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.25,0.25,0.172414,0.448276,0.416667,0.661675,0.497382,0.524435,0.633125,0.508545,0.299868,0.304633
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.00,0.75,0.586207,0.206897,0.125000,0.449453,0.562562,0.378946,0.697656,0.511100,0.377197,0.303891


In [144]:
y_train_resample.head()

0    1
1    0
2    1
3    0
4    1
Name: failure, dtype: int64

In [147]:
#Treinamento do modelo
train_pipeline.fit(X_train, y_train)

Pipeline(steps=[('transformer_pipe',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(fill_value='#',
                                                                                 strategy='constant')),
                                                                  ('encode',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['product_code',
                                                   'attribute_0',
                                                   'attribute_1']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['attribute_2', 'attribute_3',
                                                   'measurement_0',
                                                   'measurement_1',
                                                   'measurement_2',
                                                   'measurement_10',
                                                   'measurement_11',
                                                   'measurement_12',
                                                   'measurement_14',
                                                   'measurement_15',
                                                   'measurement_16',
                                                   'measurement_17'])])),
                ('dt', DecisionTreeClassifier(random_state=1))])

In [155]:
y_test_pred = train_pipeline.predict(X_test)

In [156]:
y_train_pred = train_pipeline.predict(X_train)

In [157]:
print(classification_report(y_test, y_test_pred))
cm = confusion_matrix(y_test, y_test_pred)
ConfusionMatrixDisplay(cm).plot()

ValueError: Found input variables with inconsistent numbers of samples: [21256, 5314]

In [ ]:
from sklearn.impute import SimpleImputer

## PIPELINE

- remover tais colunas
- criar outras colunas
- imputer mean nas colunas numéricas
- encoding nas categóricas
- scaler

- balanceamento de classe

# Modelo

Usa o modelo dentro do pipeline

# Avaliar o modelo

In [ ]:
def metricas_classificacao(estimator, X_train, X_test, y_train, y_test):
    
    print("\nMétricas de avaliação de treino:")

    y_pred_train = estimator.predict(X_train)
    y_probs_train = estimator.predict_proba(X_train)[:, 1]

    ConfusionMatrixDisplay.from_predictions(y_train, y_pred_train)
    plt.show()

    print(classification_report(y_train, y_pred_train))
    
    disp = RocCurveDisplay.from_predictions(y_train, y_probs_train)
    disp.ax_.set_title(f"AUC: {roc_auc_score(y_train, y_probs_train):.3f}", fontsize=16)
    x = np.linspace(0, 1, 100)
    plt.plot(x, x, ls=":", color="black")
    plt.show()

    # ============================================

    print("\nMétricas de avaliação de teste:")

    y_pred_test = estimator.predict(X_test)
    y_probs_test = estimator.predict_proba(X_test)[:, 1]

    ConfusionMatrixDisplay.from_predictions(y_test, y_pred_test)
    plt.show()

    print(classification_report(y_test, y_pred_test))
    
    disp = RocCurveDisplay.from_predictions(y_test, y_probs_test)
    disp.ax_.set_title(f"AUC: {roc_auc_score(y_test, y_probs_test):.3f}", fontsize=16)
    x = np.linspace(0, 1, 100)
    plt.plot(x, x, ls=":", color="black")
    plt.show()

In [ ]:
# TENTA ESCOLHER 3 FEATURES COM O TARGET E RODA UM MODELO.
# VEJA O QUE ACONTECE

In [ ]:
# particionar os dados -> test = validação
from sklearn.model_selection import  train_test_split


X = descricao[['measurement_1', 'measurement_2', 'measurement_3']] # são aleatórias só para exemplificar
y = descricao['failure']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# PIPELINE -> TUDO QUE ACONTECE NO TREINO, ELE VAI REPLICAR PARA O TESTE. ELE ORGANIZA OS PASSOS
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier


#dados faltantes e normalização
pipe_features_num = Pipeline([
    ('input_num',SimpleImputer(strategy='mean')), 
    ('std', StandardScaler())
]) # colocar os passos e os métodos do que iremos fazer com as features numéricas

#transformar as colunas
pre_process = ColumnTransformer([
    ('transf_num', pipe_features_num, X_train.columns.tolist())
]) # apliquei em todas as colunas


# Incluir o modelo no pipeline
pipe_rf = Pipeline([
    ('pre_process', pre_process), 
    ('rf', RandomForestClassifier(class_weight='balanced'))])

pipe_rf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, RocCurveDisplay, roc_auc_score

In [ ]:
metricas_classificacao(pipe_rf, X_train, X_test, y_train, y_test)